In [2]:
import requests,time,datetime
import sys
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import utils.DateAndTimeUtil as datu
import utils.CommonUtil as cu
import utils.StockSpider as ss

In [3]:
df_hsA = pd.read_excel("data/沪深A股.xls",converters= {u'StockCode':str})

In [14]:
df = df_hsA

In [3]:
# dir_path = 'E:/wangtao/PythonWorkSpace/SpiderSpace/Stock_Companies/data/'
# ss.getAnnualReports(dir_path,df,'2019')

ss.getStockDataByType('E:/wangtao/PythonWorkSpace/DataAnalysis/StockAnalyze/data/')
# ss.getStockDataByType('E:/PythonWorkSpace/DataAnalysis/Stock_Companies/data/')


正在获取：沪深A股...请勿关闭浏览器
第210页-第16条...

Traceback (most recent call last):
  File "E:\wangtao\PythonWorkSpace\DataAnalysis\StockAnalyze\utils\StockSpider.py", line 108, in getStockDataByType
    company = browser.find_element_by_xpath('//*[@id="table_wrapper-table"]/tbody/tr['+str(j+1)+']/td[3]/a')
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 394, in find_element_by_xpath
    return self.find_element(by=By.XPATH, value=xpath)
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 978, in find_element
    'value': value})['value']
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath

数据下载完毕，已保存到E:/wangtao/PythonWorkSpace/DataAnalysis/StockAnalyze/data/沪深A股.xls


In [15]:
# 显示所有列
pd.set_option('display.max_columns', None)
# 显示所有行
pd.set_option('display.max_rows', None)

In [16]:
df_2014 = pd.read_excel("data/2014年报.xls",converters= {u'stock_code':str})
df_2015 = pd.read_excel("data/2015年报.xls",converters= {u'stock_code':str})
df_2016 = pd.read_excel("data/2016年报.xls",converters= {u'stock_code':str})
df_2017 = pd.read_excel("data/2017年报.xls",converters= {u'stock_code':str})
df_2018 = pd.read_excel("data/2018年报.xls",converters= {u'stock_code':str})
df_2019 = pd.read_excel("data/2019年报.xls",converters= {u'stock_code':str})

In [17]:
code_list = []
reports = [df_2014,df_2015,df_2016,df_2017,df_2018,df_2019]
for df_report in reports:
    codes = []
    for index, row in df_report.iterrows():
        # 净资产收益率大于?%
        if (row['stock_code'][0] != '3') & \
            (row['avg_roe'] > 0.2) & \
           (row['total_revenue_gr'] > 0) & \
            (row['basic_eps'] > 0) & \
            (row['np_per_share'] > 0) & \
            (row['operate_cash_flow_ps'] > 0):
#             (row['operate_cash_flow_ps_gr'] > 0):
            #满足条件的加入
            codes.append(row['stock_code'])
            
    code_list.append(codes)

In [18]:
intersection = code_list[0]
# 取每年的交集
for i in range(1,len(code_list)):
    intersection = list(set(intersection).intersection(set(code_list[i])))

In [19]:
df_target = df[df['StockCode'].apply(lambda x: (x in intersection))]

In [20]:
df_target.shape

(18, 19)

In [12]:
df_target = df_target[df_target['PE_D'].apply(lambda x:(x != '未公布') &(x != '亏损') & (x != '0'))]
df_target = df_target[df_target['PE_S'].apply(lambda x:(x != '未公布') &(x != '亏损') & (x != '0'))]
# 新增市盈率中位数
df_target['PE_M'] = df_target['StockCode'].apply(lambda x:ss.getPEMedian(x))
df_target = df_target[df_target['PE_M'] != 0]

df_target['pe_rate'] = df_target.apply(lambda x:(float(x['PE_S']) / x['PE_M']),axis=1)
# df_target = df_target[df_target['pe_rate'] < 0.6]

In [13]:
df_target.shape

(64, 21)

In [13]:
# df_target.sort_values(by=['pe_rate'],ascending=True)

In [21]:
df_target

,StockCode,StockName,Price,PriceLimit,QuantityRelativeR,TurnoverRate,Trade,PB,PE_S,PE_D,EarningsPerShare,NetProfitDes,ROE,GrossProfitRate,NetAssetValuePerShare,CapitalStock,ScaleShareType,FinanceAnalize,MoneyFlowPerShare
109,603517,绝味食品,82.08,6.18%,1.39,0.61%,食品饮料 — 食品加工,11.03,62.43,91.139,0.45元,2.74亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比下降30.78%\t...,5.83%,34.32%,7.43元,6.09亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,NaN,0.91元
230,000963,华东医药,26.64,4.35%,2.24,1.39%,医药生物 — 化学制药,3.42,16.53,13.428,0.99元,17.31亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长8.45%\t...,13.12%,36.24%,7.77元,17.50亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,NaN,0.72元
276,603288,海天味业,169.65,4.04%,1.06,0.14%,食品饮料 — 食品加工,32.50,102.80,84.590,1.00元,32.53亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长18.27%\...,19.12%,42.94%,5.22元,32.40亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t超大盘股\t\t\t\t\t\t\t...,NaN,0.48元
418,600887,伊利股份,41.50,3.29%,1.13,0.67%,食品饮料 — 食品加工,9.86,36.22,33.614,0.61元,37.35亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比下降1.2%\t\...,13.28%,38.17%,4.20元,60.83亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t超大盘股\t\t\t\t\t\t\t...,NaN,0.61元
433,002841,视源股份,98.90,3.24%,1.45,0.41%,电子 — 光学光电子,13.26,40.69,60.580,0.83元,5.41亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比下降4.13%\t\...,10.53%,27.27%,7.42元,6.68亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,NaN,-0.08元
668,603983,丸美股份,68.20,2.39%,0.72,0.99%,化工 — 化学制品,9.76,52.82,50.714,0.67元,2.68亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长4.6%\t\t...,9.65%,67.55%,6.94元,4.01亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t小盘股\t\t\t\t\t\t\t\...,NaN,0.17元
792,603899,晨光文具,78.21,2.13%,0.75,0.21%,轻工制造 — 家用轻工,15.94,67.57,77.109,0.50元,4.65亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比下降1.44%\t\...,10.77%,27.07%,4.84元,9.27亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t超大盘股\t\t\t\t\t\t\t...,NaN,0.14元
820,600519,贵州茅台,1734.00,2.06%,1.44,0.22%,食品饮料 — 饮料制造,15.85,52.65,47.993,17.99元,226.02亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长13.29%...,15.34%,91.46%,109.24元,12.56亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t超大盘股\t\t\t\t\t\t\t...,NaN,10.05元
1142,603429,集友股份,41.28,1.55%,1.15,1.17%,轻工制造 — 包装印刷Ⅱ,13.55,77.16,295.690,0.07元,0.26亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比下降65.54%\t...,2.20%,39.73%,3.02元,3.80亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t中盘股\t\t\t\t\t\t\t\...,NaN,0.22元
1393,002508,老板电器,33.63,1.20%,0.98,1.00%,家用电器 — 白色家电,4.50,19.84,25.759,0.65元,6.12亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比下降8.66%\t\...,8.64%,54.82%,7.38元,9.49亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,NaN,0.43元


In [23]:
math.ceil(950/100)*100

1000